# Config

In [1]:
# import src modules
import sys
import os

sys.path.append(os.path.abspath("../src"))

# Generate Test Dataset

In [2]:
from test_dataset_generation import generate_test_dataset

/Users/Raman.Bylina/my/programming/Test Tasks/ISI test/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
generate_test_dataset()

# RAG System

## Create Faiss Index

In [32]:
from faiss_index_creation import create_faiss_index

In [33]:
create_faiss_index()

Total chunks: 132


Batches: 100%|██████████| 5/5 [00:00<00:00, 21.00it/s]

FAISS index built.


## Model's Answers testing

In [ ]:
import faiss
import google.generativeai as genai
from sentence_transformers import SentenceTransformer

from consts import *
from model_answering import answer_with_model
from private_consts import GEMINI_API_KEY

In [ ]:
# define models
genai.configure(api_key=GEMINI_API_KEY)
answering_model = genai.GenerativeModel(MODEL_NAME)
st_model = SentenceTransformer(EMBEDDING_MODEL_NAME)

# read saved data
index = faiss.read_index(DATA_FAISS_INDEX)
with open(DATA_CHUNKS_METADATA, "r", encoding="utf-8") as f:
    metadata = [line.strip() for line in f]
with open(DATA_CHUNKS_TEXT, "r", encoding="utf-8") as f:
    chunks = [line.strip() for line in f]

In [6]:
import pandas as pd
test_data = pd.read_csv(TEST_DATA_CSV)
test_data.head()

,question,answer,chunk_id,pdf_name
0,What percentage of policyholders are dissatisf...,55%,0,0a7bc290-5559-4b68-bd36-6d5834c8363a.pdf
1,What percentage of life insurers are focusing ...,34%,0,0a7bc290-5559-4b68-bd36-6d5834c8363a.pdf
2,What percentage of EMEA life and health insure...,80%,0,0a7bc290-5559-4b68-bd36-6d5834c8363a.pdf
3,What percentage of life insurers have earned b...,5%,0,0a7bc290-5559-4b68-bd36-6d5834c8363a.pdf


In [7]:
for row in test_data.itertuples(index=False):
    model_answer = answer_with_model(row.question, st_model, index, chunks, metadata, answering_model)
    print(f'model_answer: `{model_answer}`, expected answer: `{row.answer}`')

model_answer: `55% of policyholders are dissatisfied with claims and surrenders.`, expected answer: `55%`
model_answer: `34% of life insurers are focusing on AI-driven self-service tools.`, expected answer: `34%`
model_answer: `The context states that EMEA ranked customer engagement, onboarding, and marketing as a top-three impact area of Gen AI. The exact percentage is not provided in the given context.`, expected answer: `80%`
model_answer: `5%`, expected answer: `5%`
